### IE801(B) Logistics Management HW3

- Lee Kanghoon, 20203421

In [12]:
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import copy
import itertools

import gurobipy as gp
from gurobipy import GRB

### Branch-and-Price algorithm

Implement the Branch-and-Price algorithm for solving the Constrained Shortest Path Problem (CSPP)

![img](Images/HW3_image1.png)

reference : https://en.wikipedia.org/wiki/Branch_and_price

### Problem 1.

Using your Branch-and-Price code, solve the toy example considered in the class. 
![nn](./Images/HW3_image2.png)

reference : Desrosiers, Jacques, and Marco E. Lübbecke. "A primer in column generation." Column generation. Springer, Boston, MA, 2005. 1-32.

### Practice : solve the constrained shortest path problem with MIP

![nn](./Images/HW3_image3.png)

In [3]:
def solve_CSPP_example():
    
    # problem parameter
    num_nodes = 6
    num_edges = 10
    edges = [(1, 2, {'cost': 1, 'time': 10}),
            (1, 3, {'cost': 10, 'time': 3}),
            (2, 4, {'cost': 1, 'time': 1}),
            (2, 5, {'cost': 2, 'time': 3}),
            (3, 2, {'cost': 1, 'time': 2}),
            (3, 4, {'cost': 5, 'time': 7}),
            (3, 5, {'cost': 12, 'time': 3}),
            (4, 5, {'cost': 10, 'time': 1}),
            (4, 6, {'cost': 1, 'time': 7}),
            (5, 6, {'cost': 2, 'time': 2})]

    G = nx.DiGraph()
    G.add_node(num_nodes)
    G.add_edges_from(edges)

    model = gp.Model()
    model.Params.LogToConsole = 0
    variables = gp.tupledict()

    # (1.1), (1.6)
    for edge in edges:
        i, j, data = edge
        variables[i, j] = model.addVar(obj=data['cost'],
                                       vtype=GRB.BINARY)

    # (1.2) ~ (1.4)
    for n in range(1, num_nodes+1):
        term1 = sum(variables[i, j] for i, j in G.out_edges(n))
        term2 = sum(variables[i, j] for i, j in G.in_edges(n))
        equation = term1 - term2

        if n == 1:
            model.addConstr(equation == 1)
        elif n == num_nodes:
            model.addConstr(equation == -1)
        else:
            model.addConstr(equation == 0)

    # (1.5)
    equation = None
    for edge in edges:
        i, j, data = edge
        equation += data['time'] * variables[i, j]
    model.addConstr(equation <= 14)

    # optimize!
    model._vars = variables
    model.optimize()

    solution = model.getAttr('x', variables)
    value = model.objVal
    
    G = nx.DiGraph()
    G.add_node(num_nodes)

    for i, j in solution:
        if solution[(i, j)] == 1.0:
            G.add_edge(i, j)
    path = list(nx.algorithms.simple_paths.shortest_simple_paths(G, 1, 6))[0]
    print('path  : ', path)
    print('value : ', value)

In [5]:
solve_CSPP_example()

path  :  [1, 3, 2, 4, 6]
value :  13.0


### master problem

![nn](./Images/HW3_image4.png)

In [127]:
from itertools import tee

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [126]:
def check_same_path(branch_list, path):
    same_list = []
    diff_list = None
    for i in range(num_nodes):
        cur = set({})
        for branch, _ in branch_list:
            cur = cur.union({path[branch][i]})

        if len(cur) == 1:
            same_list.append(cur.pop())
        else:
            diff_list = list(cur)
            break
    return same_list, diff_list

In [198]:
def solve_subprob(num_nodes, edges, path_, path_list_, cost_list_, time_list_, cnt_,
                  variable_const_=[]):
    
    path = copy.copy(path_)
    path_list = copy.copy(path_list_)
    cost_list = copy.copy(cost_list_)
    time_list = copy.copy(time_list_)
    cnt = copy.copy(cnt_)
    variable_const = copy.copy(variable_const_)
    
    need_y = True
    if len(path_list) - 1 > 0:
        need_y = False
    
#     for path in fixed
    
    while True:
#         print("----------")
#         print("iteration.. : ", cnt)
        num_paths = len(path_list) - 1

        model = gp.Model()
        model.Params.LogToConsole = 0
        variables = gp.tupledict()

        for i in path_list[1:]:
            variables[i] = model.addVar(obj=cost_list[i])
        variables['y'] = model.addVar(obj=100)
        
        if not need_y:
            const_r = model.addConstr(sum(time_list[i] * variables[i] for i in path_list[1:]) <= 14)
            const_c = model.addConstr(sum(variables[i] for i in path_list[1:]) == 1)
        else:
            const_r = model.addConstr(sum(time_list[i] * variables[i] for i in path_list[1:]) <= 14)
            const_c = model.addConstr(sum(variables[i] for i in path_list[1:]) + variables['y'] == 1)

        model._vars = variables
        model.optimize()
        
        print(GRB.status.OPTIMAL)
        print(GRB.OPTIMAL, model.status)
        
        # if the problem is infeasible
        if model.status == 3:
            return path, path_list, cost_list, time_list, cnt, None, False
        solution = model.getAttr('x', variables)
        value = model.objVal
        
        print(solution, value)

        pi1 = const_r.Pi
        pi0 = const_c.Pi
        
        for edge in edges:
            edge[-1]['revised_cost'] = edge[-1]['cost'] - pi1 * edge[-1]['time']
        
        G = nx.DiGraph()
        G.add_node(num_nodes)
        G.add_edges_from(edges)
        
        for i, j in variable_const:
            print("here")
            print(i, j)
            remove_list = []
            for _, j_ in G.out_edges(i):
                print(i, j, j_)
                if j != j_:
                    remove_list.append((i, j_))
            for _, j_ in remove_list:
                G.remove_edge(i, j_)
                print("remove .. ", i, j_)
        
        print("removed graph!!")
        print(G.edges())
        length, path_sub = nx.algorithms.shortest_paths.weighted.single_source_bellman_ford(G, 1, 6, 'revised_cost')
        min_reduced_cost = length - pi0
        
        if abs(min_reduced_cost - 0) <= 1e-5:
            
#             print(path)
#             print(solution)
            
            branch_list = []
            for sol in solution:
                if solution[sol] > 0.0:
                    branch_list.append([sol, solution[sol]])
            break
        
        cost_sub = nx.classes.path_weight(G, path_sub, 'cost')
        time_sub = nx.classes.path_weight(G, path_sub, 'time')
        print("path sub : ", path_sub)
        path[cnt] = path_sub
        path_list.append(cnt)
        cost_list.append(cost_sub)
        time_list.append(time_sub)
        print("min_reduced_cost : ", min_reduced_cost)
        print("path : ", path)
        print("path : ", path_list)
        print("cost : ", cost_list)
        print("time : ", time_list)
        print("---- one sub prob iter ----")
        cnt += 1

    return path, path_list, cost_list, time_list, cnt, branch_list, True
    

In [199]:
def BP_CSSP(num_nodes, edges, path_={}, path_list_ = ['dummy'],
           cost_list_ = ['dummy'], time_list_ = ['dummy'], const_ = None, cnt_ = 1, child=False, variable_const=[]):

    path = copy.copy(path_)
    path_list = copy.copy(path_list_)
    cost_list = copy.copy(cost_list_)
    time_list = copy.copy(time_list_)
    cnt = cnt_
    const = const_
    variable_const = []
    
    if child:
        print(path)
        print(path_list)
        print(const)
        for key in path:
            cur_path = path[key]
            for i in range(len(const)):
                if const[i] != cur_path[i]:
                    print(key, path[key])
                    path_list.remove(key)
        print(path_list)
        
        print(list(pairwise(const)))
        variable_const = list(pairwise(const))
#         return 0
    
    info = [path, path_list, cost_list, time_list, cnt]
    
    path, path_list, cost_list, time_list, cnt, branch_list, feasible = solve_subprob(num_nodes, edges, path, 
                                                                                      path_list, cost_list, 
                                                                                      time_list, cnt, variable_const)
    
    if child:
        print(path, path_list, cost_list, time_list, cnt, branch_list, feasible)
        return 0
    if len(branch_list) == 1:
        pass
    
    else:
        same_list, diff_list = check_same_path(branch_list, path)
                
    print(same_list, diff_list)
    
    print(const)
    print("--------------------")
    print("--------------------")
    print("--------------------")
    print("--------------------")
    
    const = same_list + [diff_list[0]]
    value = BP_CSSP(num_nodes, edges, path, path_list, cost_list, time_list, const, cnt, True)
    print(value)
    
    print("--------------------")
    print("--------------------")
    print("--------------------")
    print("--------------------")
    
    const = same_list + [diff_list[1]]
    value = BP_CSSP(num_nodes, edges, path, path_list, cost_list, time_list, const, cnt, True)
    print(value)
    
    
        

In [200]:
# problem parameter
num_nodes = 6
edges = [(1, 2, {'cost': 1, 'time': 10}),
        (1, 3, {'cost': 10, 'time': 3}),
        (2, 4, {'cost': 1, 'time': 1}),
        (2, 5, {'cost': 2, 'time': 3}),
        (3, 2, {'cost': 1, 'time': 2}),
        (3, 4, {'cost': 5, 'time': 7}),
        (3, 5, {'cost': 12, 'time': 3}),
        (4, 5, {'cost': 10, 'time': 1}),
        (4, 6, {'cost': 1, 'time': 7}),
        (5, 6, {'cost': 2, 'time': 2})]

BP_CSSP(num_nodes, edges)

2
2 2
{'y': 1.0} 100.0
removed graph!!
[(1, 2), (1, 3), (2, 4), (2, 5), (3, 2), (3, 4), (3, 5), (4, 5), (4, 6), (5, 6)]
path sub :  [1, 2, 4, 6]
min_reduced_cost :  -97.0
path :  {1: [1, 2, 4, 6]}
path :  ['dummy', 1]
cost :  ['dummy', 3]
time :  ['dummy', 18]
---- one sub prob iter ----
2
2 2
{1: 0.7777777777777778, 'y': 0.22222222222222232} 24.555555555555564
removed graph!!
[(1, 2), (1, 3), (2, 4), (2, 5), (3, 2), (3, 4), (3, 5), (4, 5), (4, 6), (5, 6)]
path sub :  [1, 3, 5, 6]
min_reduced_cost :  -32.8888888888889
path :  {1: [1, 2, 4, 6], 2: [1, 3, 5, 6]}
path :  ['dummy', 1, 2]
cost :  ['dummy', 3, 24]
time :  ['dummy', 18, 8]
---- one sub prob iter ----
2
2 2
{1: 0.6, 2: 0.4, 'y': 0.0} 11.400000000000002
removed graph!!
[(1, 2), (1, 3), (2, 4), (2, 5), (3, 2), (3, 4), (3, 5), (4, 5), (4, 6), (5, 6)]
path sub :  [1, 3, 2, 5, 6]
min_reduced_cost :  -4.799999999999997
path :  {1: [1, 2, 4, 6], 2: [1, 3, 5, 6], 3: [1, 3, 2, 5, 6]}
path :  ['dummy', 1, 2, 3]
cost :  ['dummy', 3, 24, 